In [475]:
from pdb import set_trace as debug
from pandas.io.parsers import read_csv
from IPython.parallel import Client,require
from collections import Counter

In [476]:
import ujson
import gzip
import datetime
import re
import csv
import gzip

In [477]:
#--> RAW_FILE_DIR with the directory that contains google_play__main.json.gz
#--> OUT_DIR with the directory of where you want the processed files to go
#--> DEBUG whether to print out created CSV files or not
#--> COMPRESS_LEVEL gzip compression level. warning: severely impacts runtime.
#--> LINE_LIMIT how many lines to iterate through in the raw file. For debugging.
DUMP_DATE = '2015_04_07_23_12'
RAW_FILE_DIR = '/Users/catherinaxu/Desktop/Stanford/EC/MIG/MixRank/MixRank Raw'
OUT_DIR = '/Users/catherinaxu/Desktop/Stanford/EC/MIG/MixRank/MixRank Out'
DEBUG = 1
COMPRESS_LEVEL = 1
LINE_LIMIT = -1 # -1 means run for the entire file
PARALLEL = 0

In [478]:
RAW_FILE_IOSSDKSINDEX = '{}/mixrank__ios_sdks_index.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOTEST2 = '{}/mixrank__ios_sdks_info_test2.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDTOP500 = '{}/mixrank__android_top_500.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSTOP200 = '{}/mixrank__ios_top_200.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOFULL = '{}/mixrank__ios_sdks_info_full.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOTEST = '{}/mixrank__ios_sdks_info_test.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDSDKSINFO = '{}/mixrank__android_sdks_info.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFO = '{}/mixrank__ios_sdks_info.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDSDKSPERAPP = '{}/mixrank__android_sdks_per_app.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSPERAPP = '{}/mixrank__ios_sdks_per_app.json.gz'.format(RAW_FILE_DIR)
OUT_BASE = OUT_DIR+'/{}__'+DUMP_DATE+'.csv.gz'

In [479]:
def iter_json_gzip(filename,LINE_LIMIT=LINE_LIMIT,no_verify_appid_time=1):
    with gzip.open(filename,'rt') as file_iter:
        c = 0 
        #debug 
        #file_content = file_iter.read()
        #print (file_content)
        
        #iterating through lines in file
        for line in file_iter: 
            c += 1
            
            #makes sure that it does not read over the line limit
            if c > LINE_LIMIT and LINE_LIMIT > 0:
                break
                
            if isinstance(line,str):
                if len(line)>0:        
                    out = ujson.loads(line)
                    
                    #if it is the proper format
                    if isinstance(out,dict):
                        if 'app_id' in out and 'timestamp' in out \
                        or no_verify_appid_time:
                            yield out

# APPLESDKSINDEX

In [480]:
# The gzip file contains the same list twice
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPLESDKSINDEX(file_in, out_base, compress = 1):
    headers = ['sdk_id']
    out_filename = out_base.format('APPLESDKSINDEX')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json.get('sdk_id')])
    return out_filename

In [481]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSINDEX(RAW_FILE_IOSSDKSINDEX,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id
0,atvalidations
1,revmob
2,cocoapods
3,asyncimageview
4,urban-airship
5,nend
6,atfunctionalcollectionextensions
7,ziparchive
8,chartboost
9,cocosdenshion


#APPLESDKSINFOTEST2

In [482]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPLESDKSINFOTEST2(file_in, out_base, compress = 1):
    headers = ['app_id', 'sdk_list', 'rank'] 
    out_filename = out_base.format('APPLESDKSINFOTEST2')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            allsdks = json.get('sdk_list').split(",")
            for sdk in allsdks:
                csv_writer.writerow([json['app_id'], sdk, json.get('rank_highest')])
    return out_filename

In [483]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSINFOTEST2(RAW_FILE_IOSSDKSINFOTEST2,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_list,rank
0,301259483,afscrollview,NaN
1,301259483,appirater,NaN
2,301259483,aqgridview,NaN
3,301259483,crashlytics,NaN
4,301259483,facebook,NaN
5,301259483,flurry,NaN
6,301259483,google-analytics,NaN
7,301259483,googleconversiontracking,NaN
8,301259483,google-data-apis-gdata-,NaN
9,301259483,hasoffers-mobile-app-tracking-,NaN


#GOOGLETOP500

In [484]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def GOOGLETOP500(file_in, out_base, compress = 1):
    headers = ['category', 'sdk_count', 'date', 'sdk_list', 'rating_average', 'app_id'] 
    out_filename = out_base.format('GOOGLETOP500')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            if ('sdks' in json and json.get('sdks')):
                allsdks = json.get('sdks').split(";")
                for sdk in allsdks:
                    csv_writer.writerow([json['Category'], json['Num SDKs'], json['Updated Date'], sdk, json['Star Rating'], json['Package Name']])
            else:
                csv_writer.writerow([json['Category'], json['Num SDKs'], json['Updated Date'], json['sdks'], json['Star Rating'], json['Package Name']])
            
    return out_filename

In [485]:
df_debug = None
if DEBUG:
    df_debug = read_csv(GOOGLETOP500(RAW_FILE_ANDROIDTOP500,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,category,sdk_count,date,sdk_list,rating_average,app_id
0,Strategy,20,2014-07-02,Android Internal,4.6,com.supercell.clashofclans
1,Strategy,20,2014-07-02,Google Analytics,4.6,com.supercell.clashofclans
2,Strategy,20,2014-07-02,Google Play APK Expansion Library,4.6,com.supercell.clashofclans
3,Strategy,20,2014-07-02,AdColony,4.6,com.supercell.clashofclans
4,Strategy,20,2014-07-02,AdMob,4.6,com.supercell.clashofclans
5,Strategy,20,2014-07-02,Google Authorization,4.6,com.supercell.clashofclans
6,Strategy,20,2014-07-02,Google Wallet,4.6,com.supercell.clashofclans
7,Strategy,20,2014-07-02,Google Plus,4.6,com.supercell.clashofclans
8,Strategy,20,2014-07-02,Android Support Library,4.6,com.supercell.clashofclans
9,Strategy,20,2014-07-02,Chartboost,4.6,com.supercell.clashofclans


#APPLETOP200 

In [486]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPLETOP200(file_in, out_base, compress = 1):
    headers = ['app_id', 'price', 'sdk_count', 'rank_highest', 'date', 'sdk_list'] 
    out_filename = out_base.format('APPLETOP200')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            if ('sdks' in json and json.get('sdks')):
                allsdks = json.get('sdks').split(";")
                for sdk in allsdks:
                    csv_writer.writerow([json['App ID'], json['Price'], json['Number of SDKs'], json['Best Rank'], json['Updated Date'], sdk])
            else:
                csv_writer.writerow([json['App ID'], json['Price'], json['Number of SDKs'], json['Best Rank'], json['Updated Date'], json['sdks']])
    return out_filename

In [487]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLETOP200(RAW_FILE_IOSTOP200,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,price,sdk_count,rank_highest,date,sdk_list
0,529479190,0,10,1,2014-09-16,AdColony
1,529479190,0,10,1,2014-09-16,Appirater
2,529479190,0,10,1,2014-09-16,ATFunctionalCollectionExtensions
3,529479190,0,10,1,2014-09-16,ATValidations
4,529479190,0,10,1,2014-09-16,Chartboost
5,529479190,0,10,1,2014-09-16,Facebook
6,529479190,0,10,1,2014-09-16,Flurry
7,529479190,0,10,1,2014-09-16,HasOffers (Mobile App Tracking)
8,529479190,0,10,1,2014-09-16,iAd
9,529479190,0,10,1,2014-09-16,Open Graphics Library (OpenGL/EAGL)


#APPLESDKSINFOFULL

In [488]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPLESDKSINFOFULL(file_in, out_base, compress = 1):
    headers = ['app_id', 'sdk_list', 'rank_highest'] 
    out_filename = out_base.format('APPLESDKSINFOFULL')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            if ('sdk_list' in json and json.get('sdk_list')):
                allsdks = json.get('sdk_list').split(",")
                for sdk in allsdks:
                    csv_writer.writerow([json['app_id'], sdk, json['highest_rank']])
            else:
                csv_writer.writerow([json['app_id'], json['sdk_list'], json['highest_rank']])
    return out_filename

In [489]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSINFOFULL(RAW_FILE_IOSSDKSINFOFULL,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_list,rank_highest
0,452771538,asihttprequest,-1
1,452771538,facebook,-1
2,452771538,jsonkit,-1
3,452771538,mkstorekit,-1
4,452771538,oauth,-1
5,452771538,sbjson-for-objective-c,-1
6,452771538,sharekit,-1
7,379892783,aarki,-1
8,379892783,appirater,-1
9,379892783,cocoalumberjack,-1


#APPLESDKSINFOTEST

In [490]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPLESDKSINFOTEST(file_in, out_base, compress = 1):
    headers = ['rank_highest', 'app_id', 'sdk_list'] 
    out_filename = out_base.format('APPLESDKSINFOTEST')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            if ('sdk_list' in json and json.get('sdk_list')):
                allsdks = json.get('sdk_list').split(",")
                for sdk in allsdks:
                    csv_writer.writerow([json['highest_rank'], json['app_id'], sdk])
            else:
                csv_writer.writerow([json['highest_rank'], json['app_id'], json['sdk_list']])
    return out_filename

In [491]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSINFOTEST(RAW_FILE_IOSSDKSINFOTEST,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,rank_highest,app_id,sdk_list
0,-1,301259483,afscrollview
1,-1,301259483,appirater
2,-1,301259483,aqgridview
3,-1,301259483,crashlytics
4,-1,301259483,facebook
5,-1,301259483,flurry
6,-1,301259483,google-analytics
7,-1,301259483,googleconversiontracking
8,-1,301259483,google-data-apis-gdata-
9,-1,301259483,hasoffers-mobile-app-tracking-


#GOOGLESDKSINFO

In [492]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def GOOGLESDKSINFO(file_in,out_base,compress=1):
    headers = ['sdk_id','app_id', 'date']
    out_filename = out_base.format('GOOGLESDKSINFO')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            sdk_id = json.get('sdk_id');
            
            #if there are any apps that have installed this sdk
            if ('installs' in json and json.get('installs')):
                
                #looping through each of the offset keys
                for key in json['installs'].keys():
                    # looping through each of the results
                    results = json.get('installs').get(key).get('results')
                    for each_app in results:
                        obs = (sdk_id, each_app.get('pname'), each_app.get('upload_date'))
                        csv_writer.writerow(obs)
            #otherwise just record the sdk ID
            else:
                obs = (sdk_id, "", "")
                csv_writer.writerow(obs)
    return out_filename


In [493]:
df_debug = None
if DEBUG:
    df_debug = read_csv(GOOGLESDKSINFO(RAW_FILE_ANDROIDSDKSINFO,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id,app_id,date
0,zxing-zebra-crossing-,NaN,NaN
1,xml-pull-parsing,NaN,NaN
2,ksoap2,NaN,NaN
3,android-calendarview,NaN,NaN
4,apache-cordova,jp.naver.line.android,2014-07-17
5,apache-cordova,vStudio.Android.Camera360,2014-06-25
6,apache-cordova,com.amazon.mShop.android,2014-07-16
7,apache-cordova,com.unityconceptapps.tcr.kiat,2014-07-10
8,apache-cordova,com.usablenet.mobile.walgreen,2014-06-27
9,apache-cordova,org.wikipedia,2014-06-25


#APPLESDKSINFO

In [494]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def APPLESDKSINFO(file_in,out_base,compress=1):
    headers = ['sdk_id','app_id', 'date']
    out_filename = out_base.format('APPLESDKSINFO')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            sdk_id = json.get('sdk_id');
            
            #if there are any apps that have installed this sdk
            if ('installs' in json and json.get('installs')):
                
                #looping through each of the offset keys
                for key in json['installs'].keys():
                    
                    # looping through each of the results
                    results = json.get('installs').get(key).get('results')
                    for each_app in results:
                        obs = (sdk_id, each_app.get('id'), each_app.get('upload_date'))
                        csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (sdk_id, "", "")
                csv_writer.writerow(obs)
    return out_filename

In [495]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSINFO(RAW_FILE_IOSSDKSINFO,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id,app_id,date
0,atvalidations,NaN,NaN
1,revmob,NaN,NaN
2,cocoapods,NaN,NaN
3,asyncimageview,NaN,NaN
4,urban-airship,NaN,NaN
5,nend,NaN,NaN
6,atfunctionalcollectionextensions,420009108,2011-08-04
7,atfunctionalcollectionextensions,469369175,2012-06-28
8,atfunctionalcollectionextensions,573511269,2012-11-05
9,atfunctionalcollectionextensions,512939461,2012-05-24


#GOOGLESDKSPERAPP

In [496]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def GOOGLESDKSPERAPP(file_in,out_base,compress=1):
    headers = ['app_id','sdk_id', 'date']
    out_filename = out_base.format('GOOGLESDKSPERAPP')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            app_id = json.get('app_id');
            
            #if there are any apps that have installed this sdk
            if ('sdks' in json and json.get('sdks')):
                
                #looping through each of the offset keys
                for key in json['sdks'].keys():
                    # looping through each of the results
                    results = json.get('sdks').get(key).get('results')
                    for each_app in results:
                        obs = (app_id, each_app.get('slug'), each_app.get('upload_date_max'))
                        csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (app_id, "")
                csv_writer.writerow(obs)
    return out_filename

In [497]:
df_debug = None
if DEBUG:
    df_debug = read_csv(GOOGLESDKSPERAPP(RAW_FILE_ANDROIDSDKSPERAPP,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_id,date
0,com.themesfordroid.spacethemegolauncherex,android-api,2014-02-07
1,com.themesfordroid.spacethemegolauncherex,google-authorization,2014-02-07
2,com.themesfordroid.spacethemegolauncherex,google-cloud-messaging,2014-02-07
3,com.themesfordroid.spacethemegolauncherex,google-play-game-services,2014-02-07
4,com.themesfordroid.spacethemegolauncherex,google-play-location-apis,2014-02-07
5,com.themesfordroid.spacethemegolauncherex,google-play-maps,2014-02-07
6,com.themesfordroid.spacethemegolauncherex,google-play-panorama,2014-02-07
7,com.themesfordroid.spacethemegolauncherex,google-play-services,2014-02-07
8,com.themesfordroid.spacethemegolauncherex,google-plus,2014-02-07
9,com.themesfordroid.spacethemegolauncherex,google-wallet,2014-02-07


#APPLESDKSPERAPP

In [498]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def APPLESDKSPERAPP(file_in,out_base,compress=1):
    headers = ['app_id','sdk_id', 'date']
    out_filename = out_base.format('APPLESDKSPERAPP')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            app_id = json.get('app_id');
            
            #if there are any apps that have installed this sdk
            if ('sdks' in json and json.get('sdks')):
                
                #looping through each of the offset keys
                for key in json['sdks'].keys():
                    # looping through each of the results
                    results = json.get('sdks').get(key).get('results')
                    for each_app in results:
                        obs = (app_id, each_app.get('slug'), each_app.get('upload_date_max'))
                        csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (app_id, "")
                csv_writer.writerow(obs)
    return out_filename

In [499]:
df_debug = None
if DEBUG:
    df_debug = read_csv(APPLESDKSPERAPP(RAW_FILE_IOSSDKSPERAPP,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_id,date
0,301259483,afscrollview,2014-05-17
1,301259483,appirater,2014-05-17
2,301259483,aqgridview,2014-05-17
3,301259483,crashlytics,2014-05-17
4,301259483,facebook,2014-05-17
5,301259483,flurry,2014-05-17
6,301259483,google-analytics,2014-05-17
7,301259483,googleconversiontracking,2014-05-17
8,301259483,google-data-apis-gdata-,2014-05-17
9,301259483,hasoffers-mobile-app-tracking-,2014-05-17
